In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive//My Drive/Colab Notebooks/Assignment6')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# disregard this cell
#!nvidia-smi
#import os

#os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [0]:
# add stuff here to make it work in colab...

In [4]:
!python3 prepare_data.py shakespeare_input.txt shake

2020-06-01 21:02:22.903757: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Split input into 22981 sequences...
Serialized 100 sequences...
Serialized 200 sequences...
Serialized 300 sequences...
Serialized 400 sequences...
Serialized 500 sequences...
Serialized 600 sequences...
Serialized 700 sequences...
Serialized 800 sequences...
Serialized 900 sequences...
Serialized 1000 sequences...
Serialized 1100 sequences...
Serialized 1200 sequences...
Serialized 1300 sequences...
Serialized 1400 sequences...
Serialized 1500 sequences...
Serialized 1600 sequences...
Serialized 1700 sequences...
Serialized 1800 sequences...
Serialized 1900 sequences...
Serialized 2000 sequences...
Serialized 2100 sequences...
Serialized 2200 sequences...
Serialized 2300 sequences...
Serialized 2400 sequences...
Serialized 2500 sequences...
Serialized 2600 sequences...
Serialized 2700 sequences...
Serialized 2800 sequences...
Serialized 2900

In [5]:
import pickle

import tensorflow as tf
from prepare_data import parse_seq


# data
bs = 128
seq_len = 200
data = tf.data.TFRecordDataset("shake.tfrecords")
data = data.map(lambda x: parse_seq(x, seq_len))
data = data.shuffle(46000).batch(bs).repeat()

vocab = pickle.load(open("shake_vocab", mode="rb"))
vocab_size = len(vocab)
ind_to_ch = {ind: ch for (ch, ind) in vocab.items()}

print(vocab_size)

68


In [6]:
# don't run this cell lol
# it prints all of the data forever
for ind, thing in enumerate(data):
    inds = thing[0].numpy()
    print(inds)
    to_chars = "".join([ind_to_ch[ind] for ind in inds])
    #print(ind)
    print(to_chars)
    #print()
    break

[ 0 35 58 38 22 32 32 57 22  8 57 34 55 31 34 57 44 22  8 27 42 52 11 57
 35 29 49  8 42 57 52 13 62 57 19 52 32 32 52  3 22  8 27 57 42 31 64 35
 26 58 18 13 62 57 35 52  8 57 31  8 42 57 42 31 13 27 55 34 49 62 57 35
 55 31 32 32 57 22  8 57 56 64 62 13 35 57 62 49 22 27  8 45 58 67 52 62
 42 57 54 49 62 22 11 52  8 65 57  3 49 57 42 52 57 52 13 62 57 32 52  8
 27 22  8 27 57 35 34 31 64 58 56 52 57 55 49 31 62 57 34 55 49 57 62 49
 35 34 57 13  8 34 52 32 42 26 57 35 22 62 65 57 32 49 31 42 61 35 57 34
 55 49 57  3 31 64 45 58 58 36 18 38  5 14 26 58 43  8 57 28  8 34 22 52
 66 55 13 35 57 31  8 42]
<S>s
Will in that kingdom spend our following days:
Our son and daughter shall in Tyrus reign.
Lord Cerimon, we do our longing stay
To hear the rest untold: sir, lead's the way.

GOWER:
In Antiochus and


In [0]:
# model
n_h = 512
w_xh = tf.Variable(tf.initializers.glorot_uniform()([vocab_size, n_h]))
w_hh = tf.Variable(tf.initializers.glorot_uniform()([n_h, n_h]))
b_h = tf.Variable(tf.zeros([n_h]))

w_ho = tf.Variable(tf.initializers.glorot_uniform()([n_h, vocab_size]))
b_o = tf.Variable(tf.zeros([vocab_size]))

all_vars = [w_xh, w_hh, b_h, w_ho, b_o]

In [0]:
# training
# somewhat arbitrary number of steps
steps = 20*3500 // bs
opt = tf.optimizers.Adam()
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


@tf.function
def run_rnn_on_seq(seq_batch):
    with tf.GradientTape() as tape:
        state = tf.zeros([tf.shape(seq_batch)[0], n_h])
        total_loss = tf.constant(0.)

        for time_step in tf.range(tf.shape(seq_batch)[1] - 1):
            inp_here = tf.one_hot(seq_batch[:, time_step], vocab_size)
            state = tf.nn.tanh(tf.matmul(inp_here, w_xh) + tf.matmul(state, w_hh) + b_h)
            logits = tf.matmul(state, w_ho) + b_o

            loss_here = loss_fn(seq_batch[:, time_step+1], logits)
            total_loss += loss_here
            
        total_loss /= tf.cast(tf.shape(seq_batch)[1] - 1, tf.float32)
    grads = tape.gradient(total_loss, all_vars)
    
    # this is gradient clipping
    glob_norm = tf.linalg.global_norm(grads)
    grads = [g/glob_norm for g in grads]
    
    opt.apply_gradients(zip(grads, all_vars))

    return total_loss


# alternative function that, instead of summing up the loss at each time step,
# builds a "loss sequence" over time
# in principle, we could just build a list with one element per time step
# but this will not work with tf.function (tensors and python lists don't play
# together very well) so we use a thing called TensorArray
@tf.function
def run_rnn_on_seq_alternative(seq_batch):
    with tf.GradientTape() as tape:
        state = tf.zeros([tf.shape(seq_batch)[0], n_h])
        # this is where the per-time step losses will go
        losses = tf.TensorArray(tf.float32, size=tf.shape(seq_batch)[1]-1)

        for time_step in tf.range(tf.shape(seq_batch)[1] - 1):
            inp_here = tf.one_hot(seq_batch[:, time_step], vocab_size)  # batch x vocab
            state = tf.nn.tanh(tf.matmul(inp_here, w_xh) + tf.matmul(state, w_hh) + b_h)
            logits = tf.matmul(state, w_ho) + b_o

            # batch-size loss tensor for this time step
            # could still use loss_fn here as in the function above, but that would average over the
            # batch already. I would like to keep the batch axis here to show how this could
            # be used with a mask (see below). that's why this uses tf.nn.sparse...
            loss_here = tf.nn.sparse_softmax_cross_entropy_with_logits(seq_batch[:, time_step+1], logits)
            
            losses = losses.write(time_step, loss_here)
        losses = losses.stack() # put them together in a tensor, but it will be time x batch
        losses = tf.transpose(losses, [1, 0]) # not really necessary, but transpose to batch x time
        
        # if, say, we had a batch x time mask tensor, we could multiply it with the loss here...
        #losses = losses * mask
        
        total_loss = tf.reduce_mean(losses) # average over batch and time axes
            
        
    grads = tape.gradient(total_loss, all_vars)
    glob_norm = tf.linalg.global_norm(grads)
    grads = [g/glob_norm for g in grads]
    opt.apply_gradients(zip(grads, all_vars))

    return total_loss

In [9]:
for step, seqs in enumerate(data):
    xent_avg = run_rnn_on_seq(seqs)

    if not step % 200:
        print("Step: {} Loss: {}".format(step, xent_avg))
        print()
        

    if step > steps:
        break

Step: 0 Loss: 4.275782585144043

Step: 200 Loss: 2.2269699573516846

Step: 400 Loss: 2.073098659515381



In [10]:
import numpy as np

def sample(n_steps):
    state = tf.zeros([1, n_h])
    gen = [0]

    for step in range(n_steps):
        state = tf.nn.tanh(tf.matmul(tf.one_hot(gen[-1:], depth=vocab_size), w_xh) + tf.matmul(state, w_hh) + b_h)
        probs = tf.nn.softmax(tf.matmul(state, w_ho) + b_o).numpy()[0]
        #gen.append(np.argmax(probs))  # use argmax instead of choice if you want
        gen.append(np.random.choice(vocab_size, p=probs))
        if step==1:
          print(tf.shape(gen))
    return "".join([ind_to_ch[ind] for ind in gen])
        
agg = sample(2000)

tf.Tensor([3], shape=(1,), dtype=int32)


In [11]:
print(agg)

<S> ip,ather live so shepe ofee
phin! God, thas hethent yof leve, wish mis lask in tey giveny prasthy dowo the withire,
You cun on ie the, Whend cotrekild me withere o
ne
bulike he? thius in esturces yout tary; I heve fotr beavirs,
Se.
Thive I Imstof iny ota my,
That hell mextod my mandey ffith yom!

TROMA:
Of,
I trank wors ine sstyed.

First MarAsat: ny peraly -of methir dr vere the ? Is ofrengera berve
An, bart is siall sune de hend tool cermin; whele:
She I'll you thind leasse, se w ianhe brow s gote, and wist Sead

ToEgeO,
With I wall will be this tsesther ank.
Go bee comlceminnte, aspllave vingous.

VONEn:
Motle geat, if he!
Trethit theak of what hearle.

DONY:
I mes! and too teable, thould flom the hahe' gond!
Whyosw thot kell of lay sie,
S'sind chase nas ma lith han ford y oma:

ow ndee hives fer if in?-
Tompaiceu
geds in mere Dead to y cole sut te.

Lor'T forolis?

PRINCES:
Wold. Cy say wifr paraises you.

WYor JivirAs:
Tie hurt, Whing be on tirse;
AFill fon dim !irrs,?

hablan